<a href="https://colab.research.google.com/github/michel-wue/Seminar/blob/michel/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import requests as r
import pandas as pd
from pandas import json_normalize
import re
from google.colab import drive
import datetime
from os import times
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
play_data = pd.read_csv(
    '/content/drive/MyDrive/seminar/full_data3.csv'
)

In [15]:
numerical_cols = [cname for cname in play_data.columns if play_data[cname].dtype != "object"]
cardinality_cols = [cname for cname in play_data.columns if play_data[cname].dtype == "object"]
print(numerical_cols)
print(cardinality_cols)

['awayScore', 'homeScore', 'scoreValue', 'statYardage', 'clock', 'start_distance', 'start_yardlinie', 'start_yardsToEndzone', 'end_distance', 'end_yardlinie', 'end_yardsToEndzone', 'elo_score_away', 'elo_score_home', 'attendence']
['GameID', 'playID', 'quarter', 'type_text', 'team_play', 'start_down', 'end_down', 'season', 'week', 'Winner', 'Home_teamname', 'Away_teamname']


In [14]:

play_data['GameID'] = play_data['GameID'].astype(category)
play_data['playID'] = play_data['playID'].astype(str)
play_data['quarter'] = play_data['quarter'].astype(str)
play_data['start_down'] = play_data['start_down'].astype(str)
play_data['end_down'] = play_data['end_down'].astype(str)
play_data['season'] = play_data['season'].astype(str)
play_data['week'] = play_data['week'].astype(str)


In [8]:
play_data['clock'] = pd.to_timedelta(play_data['clock'])

In [19]:
play_data.isnull().sum()

GameID                  0
playID                  0
awayScore               0
homeScore               0
scoreValue              0
statYardage             0
quarter                 0
clock                   0
type_text               0
team_play               0
start_down              0
start_distance          0
start_yardlinie         0
start_yardsToEndzone    0
end_down                0
end_distance            0
end_yardlinie           0
end_yardsToEndzone      0
season                  0
week                    0
Winner                  0
Home_teamname           0
Away_teamname           0
elo_score_away          0
elo_score_home          0
attendence              0
dtype: int64

In [29]:
for index, row in play_data.iterrows():
  if row['Winner'] == 'Home':
    play_data.at[index, 'Winner']=0
  elif row['Winner'] == 'Away':
    play_data.at[index, 'Winner']=1
print(play_data.head())

      GameID        playID  awayScore  homeScore  scoreValue  statYardage  \
0  400951620   40095162037          0          0           0            0   
1  400951620   40095162052          0          0           0            4   
2  400951620   40095162073          0          0           0            0   
3  400951620  400951620102          0          0           0            0   
4  400951620  400951620124          0          0           0           33   

  quarter           clock          type_text team_play  ... end_yardlinie  \
0       1 0 days 00:15:00            Kickoff       cin  ...            75   
1       1 0 days 00:15:00               Rush       hou  ...            71   
2       1 0 days 00:14:25  Pass Incompletion       hou  ...            71   
3       1 0 days 00:14:19  Pass Incompletion       hou  ...            71   
4       1 0 days 00:14:14               Punt       hou  ...            46   

   end_yardsToEndzone  season  week Winner  Home_teamname  Away_teamname  

In [30]:
# Load dataset
data = play_data

# Separate target from predictors
y = data['Winner']
X = data.drop('Winner', axis=1)

# Train-test split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,random_state=0)
X_train.head()

,GameID,playID,awayScore,homeScore,scoreValue,statYardage,quarter,clock,type_text,team_play,...,end_distance,end_yardlinie,end_yardsToEndzone,season,week,Home_teamname,Away_teamname,elo_score_away,elo_score_home,attendence
330809,401437813,4014378132408,14,3,0,2,3,0 days 00:00:39,Rush,ten,...,10,47,47,2022,8,hou,ten,1583.04,1378.49,0.950997
319367,401437749,4014377491385,7,10,0,5,2,0 days 00:05:08,Pass Reception,cle,...,10,71,71,2022,4,atl,cle,1521.06,1448.24,0.925800
53049,400874682,4008746822959,23,24,3,29,3,0 days 00:06:29,Field Goal Good,nyg,...,10,100,65,2016,3,nyg,wsh,1446.71,1486.86,0.949915
153522,401030849,4010308491742,7,21,0,14,2,0 days 00:01:48,Pass Reception,bal,...,7,56,56,2018,8,car,bal,1588.19,1574.70,1.000881
209654,401128067,4011280673433,3,38,0,0,4,0 days 00:05:24,Kickoff,kc,...,10,75,75,2019,13,kc,oak,1431.38,1606.94,1.008391


In [28]:
for index, row in y.iterrows():
  if row['Winner'] == 'Home':
    y.at[index, 'Winner']=0
  elif row['Winner'] == 'Away':
    y.at[index, 'Winner']=1
print(y.head())

AttributeError: ignored

In [23]:
# Preprocessing numerical columns
numerical_transformer = StandardScaler() 

# Preprocessing categorical columns
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore')) 
])

# Bundle both preprocessors
preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_transformer, numerical_cols),
    ('cat', categorical_transformer, cardinality_cols)
])

In [31]:
model = xgb.XGBClassifier(n_estimators=100, max_depth=6, learning_rate=0.1)
# Bundle preprocessing and modeling code in a pipeline
complete_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

# Preprocess the raw training data and fit the model
model.fit(X_train, y_train, 
            eval_set=[(X_train, y_train), (X_valid, y_valid)], 
            early_stopping_rounds=20)

# Preprocess the raw validation data and make predictions
# preds = complete_pipeline.predict(X_valid)

# # Evaluate the model
# score = mean_absolute_error(y_valid, preds)
# print("MAE using the complete pipeline: {}".format(score))

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


ValueError: ignored

In [ ]:

model.fit(X_train, y_train, 
            eval_set=[(X_train, y_train), (X_valid, y_valid)], 
            early_stopping_rounds=20)